In [ ]:
import os
import getpass
import warnings
from langchain_core._api.deprecation import LangChainDeprecationWarning

# Suppress LangChainDeprecationWarning only
warnings.filterwarnings("ignore", category=LangChainDeprecationWarning)

# Securely input your OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("🔐 Enter your OpenAI API key (it will be hidden): ")

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory

# --- LLM Setup ---
llm = ChatOpenAI(temperature=0)

# Prompt template with memory support
# This defines how the AI should behave and formats each user input into a conversation turn
# The "system" message sets the assistant's role, and the "human" message is where input gets inserted
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Maintain context of the ongoing conversation."),
    ("human", "{input}")
])

# Create a session store
# In-memory store for chat history per user session
# This allows tracking of conversation history across multiple turns, simulating "memory"
session_store = {}

# Function to load or initialize a session's memory
# Uses ConversationBufferMemory with return_messages=True to retain full chat message formatting
def get_session_history(session_id: str):
    if session_id not in session_store:
        session_store[session_id] = ConversationBufferMemory(return_messages=True)
    return session_store[session_id].chat_memory

# Build a conversational chain that combines:
# - the prompt template
# - the language model
# - session-aware memory
# RunnableWithMessageHistory handles all of this and enables persistent context per session
chain = RunnableWithMessageHistory(
    prompt | llm,
    get_session_history=get_session_history,
)

# Simulate a conversation session (with 3 sequential questions)
# The session ID acts like a "user ID" so the memory is preserved across turns
session_id = "demo-session-001"
questions = [
    "Hi, who won the World Cup in 2018?",
    "And who was the top scorer?", # This tests whether the model remembers earlier context
    "Was that surprising to you?"
]

# Loop through each question, invoke the chain, and print both user + AI output
for q in questions:
    response = chain.invoke({"input": q}, config={"configurable": {"session_id": session_id}})
    print(f"🧑 You: {q}")
    print(f"🤖 AI: {response.content}\n")

    # Log current memory (for educational purposes)
    print("🧠 Memory log so far:")
    for msg in get_session_history(session_id).messages:
        print(f"{msg.type.title()}: {msg.content}")
    print("\n" + "-"*60 + "\n")